# Cleaning the dataset

In [82]:
import pandas as pd
import re

df = pd.read_csv('final_dataset.csv')

## Null values

For columns Furnished, Garden, Fireplace, SwimmingPool and Terrace we can assume that no information means the property does not have them.

Same with ToiletCount, except this one refers to a toilet separate from the bathroom. ShowerCount can also be zero if the property only has a bathtub for example.

SurfaceOfPlot is missing data for properties that are not houses (apartments, studio), so we can also fill it with zeros.

In [83]:
fill_with_zeros = ['Furnished','Garden','Fireplace','SwimmingPool','Terrace','ToiletCount','ShowerCount','SurfaceOfPlot']
for i in fill_with_zeros:
    df[i] = df[i].fillna(0)

## Iterative Imputer for LivingArea

### Mapping

We'll impute LivingArea based on BedroomCount, TypeOfProperty and SubtypeOfProperty (numerical version).
But first, we need to associate categorical values to numeric ones. This will help us use corr() later to understand correlation with price.

In [84]:
subtype_mapping = {
    'apartment': 1,
    'house': 2,
    'villa': 3,
    'ground_floor': 4,
    'duplex': 5,
    'apartment_block': 6,
    'flat_studio': 7,
    'penthouse': 8,
    'mixed_use_building': 9,
    'service_flat': 10,
    'kot': 11,
    'mansion': 12,
    'town_house': 13,
    'bungalow': 14,
    'loft': 15,
    'exceptional_property': 16,
    'country_cottage': 17,
    'farmhouse': 18,
    'triplex': 19,
    'chalet': 20,
    'other_property': 21,
    'manor_house': 22,
    'castle': 23,
    'pavilion': 24,
    'show_house': 25
}

df['SubtypeOfProperty_Numerical'] = df['SubtypeOfProperty'].map(subtype_mapping)

living_area_df = df[['BedroomCount','TypeOfProperty','SubtypeOfProperty_Numerical','LivingArea']]

### Iterative Imputing

In [85]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

living_area_imputer = IterativeImputer(random_state=0)
living_area_df_imputed = living_area_imputer.fit_transform(living_area_df)

living_area_df_imputed = pd.DataFrame(living_area_df_imputed)
df['LivingArea'] = living_area_df_imputed[3]

#print(df[['BedroomCount','TypeOfProperty','SubtypeOfProperty_Numerical','LivingArea','Price']].head()) # Debug

### Iterative Imputing for other data

#### BathroomCount

In [86]:
bathroom_count_df = df[['BedroomCount','BathroomCount','TypeOfProperty','SubtypeOfProperty_Numerical']]

bathroom_count_imputer = IterativeImputer(random_state=0)
bathroom_count_df_imputed = bathroom_count_imputer.fit_transform(bathroom_count_df)
bathroom_count_df_imputed = np.round(bathroom_count_df_imputed) # it's a count, we don't need decimals

bathroom_count_df_imputed = pd.DataFrame(bathroom_count_df_imputed)
df['BathroomCount'] = bathroom_count_df_imputed[1]

print(df[['BedroomCount','BathroomCount','TypeOfProperty','SubtypeOfProperty_Numerical']].head(10)) # Debug

   BedroomCount  BathroomCount  TypeOfProperty  SubtypeOfProperty_Numerical
0           1.0            1.0               2                            7
1          13.0            6.0               1                            6
2           4.0            2.0               1                            2
3           4.0            1.0               1                            2
4           2.0            0.0               2                            1
5           1.0            1.0               2                            1
6           6.0            4.0               1                            3
7           2.0            0.0               1                            2
8           3.0            2.0               2                            1
9           2.0            1.0               2                            1


#### RoomCount

(I think we should drop it, not always coherent)

In [87]:
room_count_df = df[['BedroomCount','TypeOfProperty','SubtypeOfProperty_Numerical','RoomCount']]

room_count_imputer = IterativeImputer(random_state=0)
room_count_df_imputed = room_count_imputer.fit_transform(room_count_df)
room_count_df_imputed = np.round(room_count_df_imputed)

room_count_df_imputed = pd.DataFrame(room_count_df_imputed)
df['RoomCount'] = room_count_df_imputed[3]

print(df[['BedroomCount','TypeOfProperty','SubtypeOfProperty_Numerical','RoomCount']].head()) # Debug

   BedroomCount  TypeOfProperty  SubtypeOfProperty_Numerical  RoomCount
0           1.0               2                            7        1.0
1          13.0               1                            6       31.0
2           4.0               1                            2        9.0
3           4.0               1                            2        9.0
4           2.0               2                            1        1.0


#### GardenArea

Properties without garden will have "0" as GardenArea. 
We could use IterativeImputer but all properties with garden have listed their areas so it's useless here.

In [88]:
GardenArea_df = df[['Garden','GardenArea','TypeOfProperty','SubtypeOfProperty_Numerical']]

GardenArea_df.loc[GardenArea_df['Garden'] == 0, 'GardenArea'] = 0
df['GardenArea'] = GardenArea_df['GardenArea']

### Montlhy Charges

Only two types of sale have monthly charges : residential_monthly_rent and annuity_monthly_amount. The missing data in those categories will be predicted with Iterative Imputer after everything else is filled with zeroes.

In [89]:
df.loc[df['TypeOfSale'] == 'residential_sale', 'MonthlyCharges'] = 0
df.loc[df['TypeOfSale'] == 'annuity_without_lump_sum', 'MonthlyCharges'] = 0
df.loc[df['TypeOfSale'] == 'annuity_lump_sum', 'MonthlyCharges'] = 0
df.loc[df['TypeOfSale'] == 'homes_to_build', 'MonthlyCharges'] = 0

TypeOfSale_mapping = {
    'residential_sale' : 0,   
    'annuity_without_lump_sum' : 0,       
    'annuity_lump_sum' : 0,                 
    'homes_to_build' : 0,   
    'residential_monthly_rent' : 1, 
    'annuity_monthly_amount' : 2       
}

df['TypeOfSale_Numerical'] = df['TypeOfSale'].map(TypeOfSale_mapping)
#print(df[['TypeOfSale','MonthlyCharges','TypeOfSale_Numerical']].head(20))
monthly_charges_df = df[['BedroomCount','TypeOfProperty','SubtypeOfProperty_Numerical','TypeOfSale_Numerical','MonthlyCharges']]

monthly_charges_imputer = IterativeImputer(random_state=0)
monthly_charges_df_imputed = monthly_charges_imputer.fit_transform(monthly_charges_df)

monthly_charges_df_imputed = pd.DataFrame(monthly_charges_df_imputed)
df['MonthlyCharges'] = monthly_charges_df_imputed[4]

#print(df[df['MonthlyCharges'] >= 0])

### Number of facades (mean)

In [90]:
new_value = int(df['NumberOfFacades'].mean())

df['NumberOfFacades'].fillna(new_value , inplace = True)

#df['NumberOfFacades'].head()

C:\Users\NGIRA\AppData\Local\Temp\ipykernel_17560\3860758433.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['NumberOfFacades'].fillna(new_value , inplace = True)


### PEB (before and after 1945)

In [91]:
under_1945_peb = df[df['ConstructionYear'] < 1945]
most_common_peb_class = under_1945_peb['PEB'].mode()[0] 
print(most_common_peb_class)

bellow_1945_peb = df[df['ConstructionYear'] > 1945]
most_common_peb_class_bellow = bellow_1945_peb['PEB'].mode()[0]
print(most_common_peb_class_bellow)

F
B


In [92]:
df['PEB'] = np.where(
    pd.isna(df['PEB']) & (df['ConstructionYear'] < 1945), most_common_peb_class,
    np.where(
        pd.isna(df['PEB']) & (df['ConstructionYear'] > 1945), most_common_peb_class_bellow,
        df['PEB']
    )
)

df['PEB'] = df['PEB'].fillna('F', axis=None)

In [93]:
df['PEB'].head()

0    B
1    D
2    B
3    F
4    B
Name: PEB, dtype: object

## Numerical values for other categorical values

#### Numerical values for kitchen

In [94]:
kitchen_mapping = {
    'NO_DATA' : 0,
    'NOT_INSTALLED': 1,
    'USA_UNINSTALLED': 2,
    'SEMI_EQUIPPED': 3,
    'USA_SEMI_EQUIPPED': 4,
    'INSTALLED': 5,
    'USA_INSTALLED': 6,
    'HYPER_EQUIPPED': 7,
    'USA_HYPER_EQUIPPED': 8
}

df['Kitchen'] = df['Kitchen'].fillna('NO_DATA')
df['Kitchen_Numerical'] = df['Kitchen'].map(kitchen_mapping)

#print(df[['BedroomCount','LivingArea','Price','Kitchen','Kitchen_Numerical']].head()) # Debug


#### Numerical values for state of building

In [95]:
stateOfBuilding_mapping = {
    'NO_DATA' : 0,
    'TO_RESTORE' : 1,
    'TO_RENOVATE' : 2,
    'TO_BE_DONE_UP' : 3,
    'GOOD' : 4,
    'AS_NEW' : 5,
    'JUST_RENOVATED' : 6
}
              
df['StateOfBuilding'] = df['StateOfBuilding'].fillna('NO_DATA')     
df['StateOfBuilding_Numerical'] = df['StateOfBuilding'].map(stateOfBuilding_mapping)       
        
#print(df[['BedroomCount','LivingArea','Price','StateOfBuilding','StateOfBuilding_Numerical']])    # Debug 

#### Numerical values for flooding zones

In [96]:
FloodingZone_mapping = {
    'NO_DATA' : 0,
    'NON_FLOOD_ZONE' : 1,
    'POSSIBLE_FLOOD_ZONE' : 2,
    'POSSIBLE_N_CIRCUMSCRIBED_FLOOD_ZONE' : 3,
    'POSSIBLE_N_CIRCUMSCRIBED_WATERSIDE_ZONE' : 4,
    'CIRCUMSCRIBED_WATERSIDE_ZONE' : 5,
    'RECOGNIZED_FLOOD_ZONE' : 6,
    'RECOGNIZED_N_CIRCUMSCRIBED_FLOOD_ZONE'  : 7,
    'CIRCUMSCRIBED_FLOOD_ZONE' : 8,
    'RECOGNIZED_N_CIRCUMSCRIBED_WATERSIDE_FLOOD_ZONE' : 9
}

df['FloodingZone'] = df['FloodingZone'].fillna('NO_DATA')
df['FloodingZone_Numerical'] = df['FloodingZone'].map(FloodingZone_mapping)                                     
       
#print(df[['FloodingZone','FloodingZone_Numerical']])      # Debug 

### Construction date

RUN AFTER RIDVAN'CODE !!!!!!!

In [97]:
df['ConstructionYear'] = df['ConstructionYear'].astype(str)
df['ConstructionYear'] = df['ConstructionYear'].replace('nan','NO_DATA')
print(df['ConstructionYear'] .head(20))

0      1969.0
1      1920.0
2      2008.0
3     NO_DATA
4      1972.0
5      1994.0
6      1970.0
7     NO_DATA
8      2023.0
9     NO_DATA
10     1961.0
11    NO_DATA
12     1987.0
13    NO_DATA
14     1966.0
15     1923.0
16    NO_DATA
17     1974.0
18     1974.0
19     1860.0
Name: ConstructionYear, dtype: object


In [98]:
df = df.dropna(subset=['District'])
df = df.dropna(subset=['Locality'])
df = df.dropna(subset=['Province'])
df = df.dropna(subset=['Region'])

In [99]:
df.to_csv('clean_dataset.csv', index=False)